In [1]:
import tensorflow as tf
import os
import numpy as np
from mit_eye import mit_itraker
import time
sess=tf.InteractiveSession()

In [2]:
with tf.name_scope('PlaceHolder'):
    LeftEye=tf.placeholder(tf.float32,[None,64,64,3],name='LeftEye')
    RightEye=tf.placeholder(tf.float32,[None,64,64,3],'RightEye')
    FaceOri=tf.placeholder(tf.float32,[None,64,64,3],'FaceOri')
    FaceMask=tf.placeholder(tf.float32,[None,25,25],'FaceMask')
    Y=tf.placeholder(tf.float32,[None,2],name='Y')
    LR=tf.placeholder(tf.float32,name='Learning_rate')

In [3]:
with tf.name_scope('MIT_Model'):
    model=mit_itraker(r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val',LeftEye,RightEye,FaceOri,FaceMask)
    y_pre=model.coordinate

Building graph done!
Loading file~
Load Data Done~


In [4]:
with tf.name_scope('LOSS'):
    Loss=(tf.losses.mean_squared_error(Y,y_pre))
    tf.summary.scalar('loss',Loss)
    ERR=tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_pre, Y), axis=1)))
    tf.summary.scalar('err',ERR)
with tf.name_scope('Trainer'):
    Trainer=tf.train.AdamOptimizer(LR).minimize(Loss)

with tf.name_scope('Logger'):
    init=tf.global_variables_initializer()

    writer_tr=tf.summary.FileWriter('./mylog/test',sess.graph)
    writer_te=tf.summary.FileWriter('./mylog/val')
    merge=tf.summary.merge_all()

In [5]:
data=model.data
'''
['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',
'val_eye_right', 'val_face_mask', 'val_eye_left']
'''

"\n['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',\n'val_eye_right', 'val_face_mask', 'val_eye_left']\n"

In [ ]:
sess.run(init)

In [6]:
#130 个loop用时1分钟
batch_size=128

s_time=time.time()
for i_ in range(130*60*2):
    mask=np.random.choice(range(48000),batch_size,replace=False)
    face_,face_mask_=data['train_face'][mask],data['train_face_mask'][mask]
    left_eye_,right_eye_=data['train_eye_left'][mask],data['train_eye_right'][mask]
    y_=data['train_y'][mask]
    feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,FaceMask:face_mask_,\
              Y:y_,LR:1e-3}
    sess.run(Trainer,feed_dict=feed_dict)
    if i_%5==0:
        loss_,m_=sess.run([Loss,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i_)
        #print('epoch:{},loss:{}'.format(i_,loss_))
    if i_%10==0:
        mask=np.random.choice(range(5000),batch_size,replace=False)
        face_,face_mask_=data['val_face'][mask],data['val_face_mask'][mask]
        left_eye_,right_eye_=data['val_eye_left'][mask],data['val_eye_right'][mask]
        y_=data['val_y'][mask]
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,FaceMask:face_mask_,\
                  Y:y_,LR:1e-3}
        loss_,err_,m_=sess.run([Loss,ERR,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i_)
        print('--epoch:{},loss:{},error:{}'.format(i_,loss_,err_))
e_time=time.time()
run_time=e_time-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))


--epoch:0,loss:27.136518478393555,error:6.1790924072265625
--epoch:10,loss:31.918827056884766,error:6.770254135131836
--epoch:20,loss:35.3564567565918,error:7.371053218841553
--epoch:30,loss:30.815580368041992,error:6.517378330230713
--epoch:40,loss:28.61261749267578,error:6.296273708343506
--epoch:50,loss:30.380817413330078,error:6.497344017028809
--epoch:60,loss:23.586030960083008,error:5.6741485595703125
--epoch:70,loss:22.798851013183594,error:5.490419387817383
--epoch:80,loss:23.604293823242188,error:5.693731784820557
--epoch:90,loss:16.40420150756836,error:4.991727828979492
--epoch:100,loss:17.6629638671875,error:4.811681747436523
--epoch:110,loss:19.1478271484375,error:5.172199726104736
--epoch:120,loss:15.08842945098877,error:4.7526140213012695
--epoch:130,loss:17.22911262512207,error:4.976778984069824
--epoch:140,loss:14.32543659210205,error:4.50862455368042
--epoch:150,loss:10.796058654785156,error:4.0510077476501465
--epoch:160,loss:14.448535919189453,error:4.631749153137207

--epoch:1380,loss:3.91787052154541,error:2.260032892227173
--epoch:1390,loss:5.151885032653809,error:2.6342883110046387
--epoch:1400,loss:5.039855003356934,error:2.5389413833618164
--epoch:1410,loss:7.193267822265625,error:2.932711362838745
--epoch:1420,loss:5.328804016113281,error:2.3395814895629883
--epoch:1430,loss:6.159863471984863,error:2.520535469055176
--epoch:1440,loss:3.987910270690918,error:2.3590288162231445
--epoch:1450,loss:5.636852264404297,error:2.640589475631714
--epoch:1460,loss:5.669020652770996,error:2.6819677352905273
--epoch:1470,loss:4.816355228424072,error:2.4883170127868652
--epoch:1480,loss:4.701507568359375,error:2.4798686504364014
--epoch:1490,loss:4.8813066482543945,error:2.3613016605377197
--epoch:1500,loss:4.927919864654541,error:2.4725306034088135
--epoch:1510,loss:4.126481056213379,error:2.368929386138916
--epoch:1520,loss:4.135992050170898,error:2.3598787784576416
--epoch:1530,loss:4.1804304122924805,error:2.3830208778381348
--epoch:1540,loss:4.58639621

--epoch:2730,loss:4.8778486251831055,error:2.533916473388672
--epoch:2740,loss:4.105621814727783,error:2.365560531616211
--epoch:2750,loss:4.394479751586914,error:2.2101001739501953
--epoch:2760,loss:3.205439329147339,error:2.083155870437622
--epoch:2770,loss:5.276622772216797,error:2.521131992340088
--epoch:2780,loss:5.026275157928467,error:2.4435224533081055
--epoch:2790,loss:5.114863395690918,error:2.3631882667541504
--epoch:2800,loss:4.053992748260498,error:2.212127447128296
--epoch:2810,loss:4.558663368225098,error:2.303661346435547
--epoch:2820,loss:3.604247570037842,error:2.163069248199463
--epoch:2830,loss:4.013731956481934,error:2.2049076557159424
--epoch:2840,loss:3.793912172317505,error:2.139176845550537
--epoch:2850,loss:3.331587314605713,error:2.0605533123016357
--epoch:2860,loss:3.941960096359253,error:2.221745252609253
--epoch:2870,loss:3.5816359519958496,error:2.0894689559936523
--epoch:2880,loss:3.4407200813293457,error:2.2442920207977295
--epoch:2890,loss:3.5692231655

--epoch:4080,loss:2.997426986694336,error:1.914297342300415
--epoch:4090,loss:4.518852710723877,error:2.12233829498291
--epoch:4100,loss:3.674536943435669,error:2.0742363929748535
--epoch:4110,loss:2.8977513313293457,error:1.9393601417541504
--epoch:4120,loss:4.540942192077637,error:2.358187675476074
--epoch:4130,loss:3.4348397254943848,error:2.030942440032959
--epoch:4140,loss:2.96455454826355,error:1.8709460496902466
--epoch:4150,loss:2.878512382507324,error:1.894373893737793
--epoch:4160,loss:2.0413095951080322,error:1.7154192924499512
--epoch:4170,loss:2.995654821395874,error:1.8960344791412354
--epoch:4180,loss:3.7391157150268555,error:2.033409357070923
--epoch:4190,loss:2.8346590995788574,error:1.9326536655426025
--epoch:4200,loss:3.7833592891693115,error:1.979119062423706
--epoch:4210,loss:3.8623063564300537,error:2.150162696838379
--epoch:4220,loss:4.525166034698486,error:2.178330421447754
--epoch:4230,loss:2.108043670654297,error:1.666252613067627
--epoch:4240,loss:4.075153827

--epoch:5430,loss:2.9236557483673096,error:1.928246259689331
--epoch:5440,loss:3.0938894748687744,error:1.8661051988601685
--epoch:5450,loss:2.7212302684783936,error:1.7277519702911377
--epoch:5460,loss:3.9775187969207764,error:2.0839905738830566
--epoch:5470,loss:3.2581675052642822,error:1.9929084777832031
--epoch:5480,loss:3.3323333263397217,error:2.0467023849487305
--epoch:5490,loss:2.2467050552368164,error:1.7690234184265137
--epoch:5500,loss:2.5388238430023193,error:1.8078269958496094
--epoch:5510,loss:4.412941932678223,error:2.169919729232788
--epoch:5520,loss:3.5263020992279053,error:1.9960440397262573
--epoch:5530,loss:2.7602880001068115,error:1.8128435611724854
--epoch:5540,loss:3.4659063816070557,error:1.8136286735534668
--epoch:5550,loss:3.09009051322937,error:1.833456039428711
--epoch:5560,loss:4.5859694480896,error:2.2177083492279053
--epoch:5570,loss:3.566093683242798,error:2.011925458908081
--epoch:5580,loss:2.6643545627593994,error:1.9579663276672363
--epoch:5590,loss:2

...

--epoch:9470,loss:1.7120546102523804,error:1.4049639701843262
--epoch:9480,loss:2.2384121417999268,error:1.7558265924453735
--epoch:9490,loss:2.7543013095855713,error:1.631989598274231
--epoch:9500,loss:3.0632283687591553,error:1.8696818351745605
--epoch:9510,loss:2.517573356628418,error:1.6607038974761963
--epoch:9520,loss:2.9796934127807617,error:1.7234034538269043
--epoch:9530,loss:2.0919606685638428,error:1.7161626815795898
--epoch:9540,loss:3.953909397125244,error:1.82575261592865
--epoch:9550,loss:1.8813544511795044,error:1.5435749292373657
--epoch:9560,loss:3.029569149017334,error:1.8888204097747803
--epoch:9570,loss:3.684480667114258,error:1.8905208110809326
--epoch:9580,loss:2.0904030799865723,error:1.5934349298477173
--epoch:9590,loss:3.0964603424072266,error:1.8458154201507568
--epoch:9600,loss:2.2163069248199463,error:1.584636926651001
--epoch:9610,loss:3.248422145843506,error:1.875582218170166
--epoch:9620,loss:2.9970145225524902,error:1.5926282405853271
--epoch:9630,loss:

--epoch:10800,loss:2.6126646995544434,error:1.781358242034912
--epoch:10810,loss:3.678279399871826,error:1.9889521598815918
--epoch:10820,loss:2.664525032043457,error:1.6947003602981567
--epoch:10830,loss:3.224123477935791,error:1.681578516960144
--epoch:10840,loss:3.7000977993011475,error:1.9601292610168457
--epoch:10850,loss:2.823768138885498,error:1.669395923614502
--epoch:10860,loss:2.485441207885742,error:1.7277753353118896
--epoch:10870,loss:2.734450101852417,error:1.5921244621276855
--epoch:10880,loss:3.6196134090423584,error:1.9301064014434814
--epoch:10890,loss:2.3830628395080566,error:1.6286556720733643
--epoch:10900,loss:3.442458391189575,error:1.8857941627502441
--epoch:10910,loss:2.6182737350463867,error:1.7460838556289673
--epoch:10920,loss:3.0954012870788574,error:1.8240145444869995
--epoch:10930,loss:2.745741844177246,error:1.929032564163208
--epoch:10940,loss:1.9031569957733154,error:1.5522887706756592
--epoch:10950,loss:3.2923264503479004,error:1.9257746934890747
--ep

--epoch:12120,loss:2.097235679626465,error:1.567216396331787
--epoch:12130,loss:2.2650692462921143,error:1.6373517513275146
--epoch:12140,loss:2.749580144882202,error:1.7174323797225952
--epoch:12150,loss:3.15579891204834,error:1.6834502220153809
--epoch:12160,loss:2.8297526836395264,error:1.7789833545684814
--epoch:12170,loss:2.7307677268981934,error:1.6623632907867432
--epoch:12180,loss:2.3204381465911865,error:1.7804433107376099
--epoch:12190,loss:2.9470906257629395,error:1.6897985935211182
--epoch:12200,loss:2.516090154647827,error:1.741302251815796
--epoch:12210,loss:2.668416976928711,error:1.7244755029678345
--epoch:12220,loss:2.8486263751983643,error:1.712468147277832
--epoch:12230,loss:2.7724673748016357,error:1.800921082496643
--epoch:12240,loss:2.5926425457000732,error:1.7727177143096924
--epoch:12250,loss:2.498537302017212,error:1.808867335319519
--epoch:12260,loss:2.4730353355407715,error:1.6780171394348145
--epoch:12270,loss:2.775667667388916,error:1.7283978462219238
--epo

--epoch:13440,loss:2.570610284805298,error:1.6666394472122192
--epoch:13450,loss:1.9494847059249878,error:1.5724530220031738
--epoch:13460,loss:4.604611873626709,error:2.094303607940674
--epoch:13470,loss:2.014049768447876,error:1.6316800117492676
--epoch:13480,loss:4.157932281494141,error:1.9086238145828247
--epoch:13490,loss:2.2929961681365967,error:1.592179298400879
--epoch:13500,loss:2.059417963027954,error:1.6473344564437866
--epoch:13510,loss:3.45650053024292,error:1.7783697843551636
--epoch:13520,loss:2.3115227222442627,error:1.7428865432739258
--epoch:13530,loss:3.0625529289245605,error:1.7699432373046875
--epoch:13540,loss:2.6458468437194824,error:1.7823399305343628
--epoch:13550,loss:3.896026849746704,error:2.0435025691986084
--epoch:13560,loss:4.466744899749756,error:1.9238224029541016
--epoch:13570,loss:3.223273515701294,error:1.8644236326217651
--epoch:13580,loss:2.515974521636963,error:1.6312229633331299
--epoch:13590,loss:3.802332878112793,error:1.9450695514678955
--epoc

--epoch:14760,loss:2.8572773933410645,error:1.785832405090332
--epoch:14770,loss:1.870628833770752,error:1.5659549236297607
--epoch:14780,loss:1.9346561431884766,error:1.5264089107513428
--epoch:14790,loss:4.250275135040283,error:1.8584272861480713
--epoch:14800,loss:3.669609785079956,error:1.9042551517486572
--epoch:14810,loss:4.335222244262695,error:1.9748650789260864
--epoch:14820,loss:2.874145746231079,error:1.7064268589019775
--epoch:14830,loss:4.31540060043335,error:1.8675811290740967
--epoch:14840,loss:2.065678119659424,error:1.5941232442855835
--epoch:14850,loss:2.8780574798583984,error:1.720125675201416
--epoch:14860,loss:2.0184364318847656,error:1.554882287979126
--epoch:14870,loss:2.018056631088257,error:1.5440597534179688
--epoch:14880,loss:2.8565921783447266,error:1.8514426946640015
--epoch:14890,loss:2.036625385284424,error:1.5264065265655518
--epoch:14900,loss:2.9401297569274902,error:1.6226582527160645
--epoch:14910,loss:3.645538568496704,error:1.720943808555603
--epoch

In [25]:
#run_time=time.time()-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))

Trainning Time:1 h,59 min,19.34 sec


In [9]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model_err17.ckpt',global_step=130*60*3)

'./model_save/model_err17.ckpt-23400'

In [ ]:
#加载模型
saver=tf.train.Saver()
saver.restore(sess,'./model_save/model.ckpt-23400')

In [ ]:
batch_size=128
mask=np.random.choice(range(5000),batch_size,replace=False)
face_,face_mask_=data['val_face'][mask],data['val_face_mask'][mask]
left_eye_,right_eye_=data['val_eye_left'][mask],data['val_eye_right'][mask]
y_=data['val_y'][mask]
feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,FaceMask:face_mask_,\
          Y:y_,LR:1e-3}
loss_,m_=sess.run([Loss,merge],feed_dict=feed_dict)

In [ ]:
data={}
mean_val=[]
file_addr=r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val'
for i in [
          'val_eye_left.npy',
          'val_eye_right.npy',
          'val_face.npy',
          'val_face_mask.npy',
          'val_y.npy']:
    d_=np.load(os.path.join(file_addr,i))
    ori_shape=d_.shape
    d_=d_.reshape(ori_shape[0],-1).astype('float32')
    if i[-5] !='y':
        #使得所有值小于 1
        d_/=255.0
        #使得均值为0
        mean_=np.mean(d_,axis=0)
        d_-=mean_
        mean_val.append(mean_)
    data[i[:-4]]=d_.reshape(ori_shape)
print('Load Data Done~')

In [ ]:
data['val_y'].shape

In [ ]:
np.min(data['val_y'][:,0]),np.min(data['val_y'][:,1])

In [ ]:
np.max(data['val_y'][:,0]),np.max(data['val_y'][:,1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8,7), dpi=5000)
plt.scatter(data['val_y'][:,0],data['val_y'][:,1],10)
plt.plot()
plt.show()